In [0]:
!wget https://zenodo.org/api/files/c8f9b6fe-82ac-481c-ad9c-12b5581cb4b4/Audio_Song_Actors_01-24.zip
!wget https://zenodo.org/api/files/c8f9b6fe-82ac-481c-ad9c-12b5581cb4b4/Audio_Speech_Actors_01-24.zip

!unzip -q -d ./song Audio_Song_Actors_01-24.zip 
!unzip -q -d ./speech Audio_Speech_Actors_01-24.zip 


Modality (01 = full-AV, 02 = video-only, 03 = audio-only).

Vocal channel (01 = speech, 02 = song).

Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).

Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the 'neutral' emotion.

Statement (01 = "Kids are talking by the door", 02 = "Dogs are sitting by the door").

Repetition (01 = 1st repetition, 02 = 2nd repetition).

Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).

### here you need to download files from github repo, I did not add it since I sync my local to colab

In [0]:
!unzip -q Audio-Visual-Emotion-and-Sentiment-Research-audio.zip
!mv Audio-Visual-Emotion-and-Sentiment-Research-audio ./Audio-Visual-Emotion-and-Sentiment-Research/ 

In [0]:

# %% codecell
#  pydub is required and not in colab already
!pip install soundfile
# %% codecell
# get model files and place in assets folder
!wget https://max-assets-prod.s3.us-south.cloud-object-storage.appdomain.cloud/max-audio-classifier/1.0.0/assets.tar.gz
!tar -xzvf assets.tar.gz
!mv classifier_model.h5 ./Audio-Visual-Emotion-and-Sentiment-Research/assets/
!mv vggish_pca_params.npz ./Audio-Visual-Emotion-and-Sentiment-Research/assets/
!mv vggish_model.ckpt ./Audio-Visual-Emotion-and-Sentiment-Research/assets/


In [33]:
%tensorflow_version 1.x
# %cd ./Audio-Visual-Emotion-and-Sentiment-Research/
import os
import sys

module_path = os.path.abspath(os.path.join('./Audio-Visual-Emotion-and-Sentiment-Research/Scripts'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)


/content/Audio-Visual-Emotion-and-Sentiment-Research
/content/Audio-Visual-Emotion-and-Sentiment-Research/Scripts


In [34]:
# %% codecell
import tensorflow as tf
from models_api import VggishModelWrapper
import pre_process_func

Vgg=VggishModelWrapper()


INFO:tensorflow:Restoring parameters from assets/vggish_model.ckpt


In [0]:

import glob


file_list=glob.glob("./speech/**/*.wav")+glob.glob("./song/**/*.wav")

raw={}
post={}
for wavFile in file_list:

    sound = pre_process_func.pre_process(wavFile)
    raw_embeddings,post_processed_embed =Vgg.generate_embeddings(sound)
    filename=wavFile.split("/")[-1]
    raw[filename]=raw_embeddings
    post[filename]=post_processed_embed
    



In [0]:
import pickle
embeddings={"raw":raw,"post":post}

with open('embeddings.dat', 'wb') as outfile:
    pickle.dump(embeddings, outfile, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
import pickle
with open('embeddings.dat', 'rb') as file:
    embeddings=pickle.load(file)
